https://stackoverflow.com/questions/12761510/python-how-can-i-calculate-the-average-word-length-in-a-sentence-using-the-spl

In [1]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
ran.seed(5125)

%load_ext cython
%load_ext cythonmagic
%matplotlib inline

from pandas_confusion import ConfusionMatrix
from scipy.cluster.vq import whiten

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *

/usr/local/lib/python3.4/dist-packages/IPython/extensions/cythonmagic.py:21: UserWarning: The Cython magic has been moved to the Cython package
  warnings.warn("""The Cython magic has been moved to the Cython package""")
/usr/local/lib/python3.4/dist-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/usr/local/lib/python3.4/dist-packages/sklearn/qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


In [2]:
data = pd.read_csv("data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head(n=1)

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [3]:
word = "likes the sound of thunder."
print(sum(c != ' ' for c in word))
#data["Lenght_of_Rows_withSpaces"] = None
data["Lenght_of_Rows_withoutSpaces"] = np.nan

data.head(n=1)

23


,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,...,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY,Lenght_of_Rows_withoutSpaces
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,...,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1,NaN


### Average word length in a sentence

In [17]:
# df1['e'] = Series(np.random.randn(sLength), index=df1.index) .loc[:,'f']
for index, sentence in data.iterrows():
    filtered = ''.join(filter(lambda x: x not in '".,;!?-', sentence["STATUS"]))
    words = [word for word in filtered.split() if word]
    try:
        avg = sum(map(len, words))/len(words)
    except ZeroDivisionError:
        avg = 0
    #print("%0.1f" % avg)

In [ ]:
for index, sentence in data.iterrows():
    #var1 = data[["STATUS"]][sentence.index]
    data["Lenght_of_Rows_withoutSpaces"][sentence.index] = sum(c != ' ' for c in sentence["STATUS"])
    data["Lenght_of_Rows_withSpaces"][index] = len(sentence["STATUS"])

data.head(n=2)

In [175]:
d = {'n': "no", 'y': "yes"}

# http://stackoverflow.com/a/17702781
data = data.replace(d)

neu = data[["#AUTHID","STATUS","cNEU"]]
ext = data[["#AUTHID","STATUS","cEXT"]]
agr = data[["#AUTHID","STATUS","cAGR"]]
con = data[["#AUTHID","STATUS","cCON"]]
opn = data[["#AUTHID","STATUS","cOPN"]]

In [176]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66)
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66)
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66)
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66)
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66)

In [177]:
for sentence in data: 
    print(sum(c != ' ' for c in sentence[[2]]))
#data[["STATUS"]]

TypeError: string indices must be integers

### Lexical and other features

In [ ]:
def SyntacticFeatures(chapters):
    """
    Extract feature vector for part of speech frequencies
    """
    def token_to_pos(ch):
        tokens = n.word_tokenize(ch)
        return [p[1] for p in n.pos_tag(tokens)]

    chapters_pos = [token_to_pos(ch) for ch in chapters]
    pos_list = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS']
    fvs_syntax = np.array([[ch.count(pos) for pos in pos_list]
                           for ch in chapters_pos]).astype(np.float64)

    # normalise by dividing each row by number of tokens in the chapter
    fvs_syntax /= np.c_[np.array([len(ch) for ch in chapters_pos])]

    return fvs_syntax

feature_sets.append(SyntacticFeatures(neu[["STATUS"]]))
feature_sets

In [ ]:
def LexicalFeatures(chapters):

    sentence_tokenizer = n.data.load('tokenizers/punkt/english.pickle')
    word_tokenizer = n.tokenize.RegexpTokenizer(r'\w+')
    
    for row in chapters.iterrows():
        # print(row)
        
        num_chapters = len(row)
        tokens = n.word_tokenize(row.lower())
        words = word_tokenizer.tokenize(row.lower())
        sentences = sentence_tokenizer.tokenize(row)
        vocab = set(words)
        words_per_sentence = np.array([len(word_tokenizer.tokenize(s))
                                           for s in sentences])
        print("druhy pokus:", num_chapters, tokens, words, sentences, vocab, words_per_sentence)

        # create an array of zeros https://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.zeros.html
        #fvs_lexical = np.zeros((len(chapters), 3), np.float64)
        #fvs_punct = np.zeros((len(chapters), 3), np.float64)

        #print("testing:", num_chapters, chapters, fvs_lexical, fvs_punct)

#         for e, ch_text in enumerate(num_chapters):
            # note: the nltk.word_tokenize includes punctuation
            

#             # average number of words per sentence
#             fvs_lexical[e, 0] = words_per_sentence.mean()
#             # sentence length variation
#             fvs_lexical[e, 1] = words_per_sentence.std()
#             # Lexical diversity
#             fvs_lexical[e, 2] = len(vocab) / float(len(words))

#             # Commas per sentence
#             fvs_punct[e, 0] = tokens.count(',') / float(len(sentences))
#             # Semicolons per sentence
#             fvs_punct[e, 1] = tokens.count(';') / float(len(sentences))
#             # Colons per sentence
#             fvs_punct[e, 2] = tokens.count(':') / float(len(sentences))


        # apply whitening to decorrelate the features
#         fvs_lexical = whiten(fvs_lexical)
#         fvs_punct = whiten(fvs_punct)

#     return fvs_lexical, fvs_punct

In [ ]:
#LexicalFeatures(neu[["STATUS"]])
word_tokenizer = n.tokenize.RegexpTokenizer(r'\w+')
words_per_sentence_AVG = word_tokenizer.tokenize(neu["STATUS"].str.lower())
feature_sets = None
feature_sets = list(LexicalFeatures(neu[["STATUS"]]))
feature_sets